#### This file can be used to reproduce the Acadia related part in Table 2, 3, 7 and Table 4 in the paper. 
This file reads data directly from Github. If you would like to use the Trip Construction file to generate your own output in Binder, simply change the *pmatrix_url* in Chunk[5] to the one under option 2 and run the chunks below.

In [2]:
import pandas as pd
import numpy as np
import googlemaps
import re
import string
import scipy
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [3]:
def split_date(df):
    df['datetaken'] = pd.to_datetime(df['datetaken'])
    df['date'] = [d.date() for d in df['datetaken']]
    df['year'] = pd.DatetimeIndex(df['date']).year
    df['month'] = pd.DatetimeIndex(df['date']).month
    return df

def subset_data(input,month):
    subset = input[input['month'] == month]
    return subset

In [23]:
path = "../Data/"
park = 'acadia'
input_url = path+park+"_NP_cluster.csv"
position_url = path+park+"_NP_coords.csv"
dist_matrix_url = path+park+"_NP_dist_matrix.csv"

# load the Flickr photo with cluster information
df = pd.read_csv(input_url)
df = split_date(df) 

In [5]:
df.head(n=5)

,index,id,owner,datetaken,latitude,longitude,title,accuracy,views,Cluster,date,year,month
0,0,8918787381,74212514@N04,2010-01-10 15:50:46,44.354492,-68.051204,Acadia National Park,12.0,793,0,2010-01-10,2010,1
1,1,29498596186,74212514@N04,2010-01-10 16:03:20,44.354492,-68.051204,Maine - Acadia National Park,12.0,5829,0,2010-01-10,2010,1
2,2,8919396564,74212514@N04,2010-01-10 16:15:59,44.354492,-68.051204,DSC03484,12.0,55,0,2010-01-10,2010,1
3,3,8918780331,74212514@N04,2010-01-10 16:31:06,44.354492,-68.051204,DSC03491,12.0,57,0,2010-01-10,2010,1
4,4,8918778905,74212514@N04,2010-01-10 16:42:40,44.354492,-68.051204,DSC03498,12.0,67,0,2010-01-10,2010,1


In [6]:
# load the position of attractions
position = pd.read_csv(position_url) 
position['coord'] = list(zip(position.Latitude, position.Longitude))
places = position['Clusters from Data'].values

In [8]:
places

array(['Schoodic Institute', 'Bass Harbor', 'Southwest Harbor',
       'Northeast Harbor', 'Bar Harbor', 'Wild Gardens of Acadia',
       'Cadillac Mountain', 'Penobscot Peak', 'Bubble Rock',
       'Jordan Pond', 'Boulder Beach', 'Thunder Hole', 'Sand Beach'],
      dtype=object)

In [9]:
# load the distance matrix
dist_matrix = pd.read_csv(dist_matrix_url,index_col=0)
dist_matrix.head()

,Schoodic Institute,Bass Harbor,Southwest Harbor,Northeast Harbor,Bar Harbor,Wild Gardens of Acadia,Cadillac Mountain,Penobscot Peak,Bubble Rock,Jordan Pond,Boulder Beach,Thunder Hole,Sand Beach
Schoodic Institute,0.000,81.026,74.132,73.382,73.512,77.424,84.215,69.457,82.548,85.141,82.609,83.429,84.342
Bass Harbor,84.206,0.000,8.080,26.813,29.721,32.390,34.466,22.888,32.798,35.392,37.576,38.395,39.309
Southwest Harbor,77.312,8.080,0.000,19.919,22.828,25.497,27.572,15.994,25.905,28.498,30.682,31.502,32.415
Northeast Harbor,76.562,26.813,19.919,0.000,17.660,14.353,22.404,3.925,20.737,23.330,14.331,15.151,16.064
Bar Harbor,76.747,29.791,22.898,17.730,0.000,4.416,10.117,13.805,8.450,11.043,9.602,10.422,11.335


In [10]:
# load probability matrix for the observed trips in the park
# option 1: read data directly from Github
# pmatrix_url = path+park+"_pmatrix/"+park+"_NP_cluster_prob_matrix_"

# option 2: read data from Binder when you generate your own output from the Trip Construction file
pmatrix_url = path+park+"_pmatrix/"+park+"_NP_cluster_prob_matrix_"
pmatrix_all = [pmatrix_url+str(i)+".csv" for i in range(1,13)]
pmatrix_summer = [pmatrix_url+str(i)+".csv" for i in range(5,10)]
pmatrix_non_summer = [pmatrix_url+str(i)+".csv" for i in range(1,5)]+[pmatrix_url+str(i)+".csv" for i in range(10,13)]

In [ ]:
print('pmatrix_all',"\n",pmatrix_all)
print("pmatrix_summer","\n",pmatrix_summer)
print("pmatrix_non_summer","\n",pmatrix_non_summer)

### Construct distance matrix and attractiveness matrix needed for the model
here should change to the open source package and change the function

In [ ]:
# Calculate travel distance (in km) using [osm](https://github.com/Project-OSRM/osrm-backend/blob/master/docs/http.md#:~:text=52.51717%0A%20%20%20%20%20%20%20%20%20%5D%2C%0A%20%20%20%20%20%20%7D%0A%20%20%20%5D%2C%0A%20%20%20%22code%22%20%3A%20%22Ok%22%0A%7D-,Route%20service,-Finds%20the%20fastest)
def get_dist_matrix(df):
    destinations = df.coord
    names = df['Clusters from Data'].values    
    dim = len(destinations)
    dist_matrix = np.zeros((dim, dim), float)        
    for i in range(dim):
        actual_distance = []
        origin = destinations[i]        
        for destination in destinations:
            result = gmaps.distance_matrix(origin, destination, mode='driving')['rows'][0]['elements'][0]['distance']['value']
            result = result/1000
            actual_distance.append(result)
        dist_matrix[i] = actual_distance       
    res = pd.DataFrame(data=dist_matrix, index = names, columns=names)
    return res

dist_matrix = get_dist_matrix(position)

In [ ]:
# generate attractiveness matrix, val indicates different measurements of attractiveness
def attr_matrix(df, month, val):
    attr_matrix = pd.DataFrame()
    df = subset_data(df, month)    
    attr_matrix['Places'] = position['Clusters from Data'].values
    # here it constructs the attractiveness using the photo views
    attr_matrix['photo_views'] = df.groupby(['Cluster'])['views'].agg('sum')
    attr_matrix['num_uploaders'] = df.groupby(['Cluster'])['owner'].nunique()
    attr_matrix['num_of_photos'] = df.groupby(['Cluster']).size()
    attr_matrix['avg_view_per_user'] = attr_matrix['photo_views']/attr_matrix['num_uploaders']
    # here it selects different measurements of attractiveness
    if val == 1:
        attr_matrix['total_attr'] = attr_matrix['num_of_photos'] #Aj1
    if val == 2:
        attr_matrix['total_attr'] = attr_matrix['num_uploaders'] #Aj2
    if val == 3:
        attr_matrix['total_attr'] = attr_matrix['num_of_photos'] * attr_matrix['avg_view_per_user'] #Aj3  
    attr_matrix = attr_matrix.fillna(0)
    attr_matrix['total_attr_log'] = np.log(attr_matrix['total_attr']+1)
    attr_matrix = attr_matrix.set_index('Places')
    return attr_matrix

In [ ]:
# to include the neighboring effect
# select K neighbors
def neighbors(dest, dist_matrix, K):
    destinations = dist_matrix.index.values
    dist_tp = np.transpose(dist_matrix)
    neighbors = dist_tp.nsmallest(10, [dest])[1:K+1].index.values   
    return neighbors

In [45]:
# calculate centrality score based on K neighbors, attraction matrix and distance matrix
def centrality(dest, attr_matrix, K):
    neighbor_lst = neighbors(dest, dist_matrix, K)
    c = 0
    dist = 0
    for p in neighbor_lst:
        c += attr_matrix.loc[p]['total_attr_log']/dist_matrix.loc[dest][p]
        dist += dist_matrix.loc[dest][p]
        c = c/dist
    return c

### Ordinary Least Squares (OLS) Calibration

In [46]:
def getComplement(item, lst):
    results = []
    for num in lst:
        if num != item: 
            results.append(num)
    return results

# OLS dependent variable
def read_actual(pmatrix, origin):
    num = 0
    denom = 0
    result = []
    places = position['Clusters from Data'].values
    dests = getComplement(origin, places)   
    actual_pmatrix = pd.read_csv(pmatrix, index_col=0)
    for i in range(len(dests)):
        num = actual_pmatrix.loc[origin].values[i]
        denom = np.mean(actual_pmatrix.loc[origin])
        result.append(num/denom)
    return result

# OLS independent variables
# attractiveness (including Social Influence), distance, centrality, without temporal factor
def log_transform_x_NT(origin,K):
    X1, X2, X3 = [],[],[]
    total_centrality = 0
    places = position['Clusters from Data'].values
    dests = getComplement(origin, places)
    attr_mat = attr_matrix_all(df)     
    for dest in dests:
        total_centrality += centrality(dest, attr_mat, K)
        X1.append(attr_mat.loc[dest]['total_attr_log']/np.mean(attr_mat['total_attr_log']))
        X2.append(dist_matrix.loc[origin][dest]/ np.mean(dist_matrix.loc[origin]))
        X3.append(centrality(dest, attr_mat, K)/(total_centrality/len(dests)))
    var_table = pd.DataFrame()
    X1 = [x + 1 for x in X1]
    X3 = [x + 1 for x in X3]
    var_table['x1'] = np.nan_to_num(np.log(X1))
    var_table['x2'] = np.nan_to_num(np.log(X2))
    var_table['x3'] = np.nan_to_num(np.log(X3))
    return var_table

# OLS independent variables
# val indicates different measurements of attractiveness
def log_transform_x(origin,K,month,val):
    X1, X2, X3 = [],[],[]
    total_centrality = 0
    places = position['Clusters from Data'].values
    dests = getComplement(origin, places)    
    attr_mat = attr_matrix(df, month, val)
    for dest in dests:
        total_centrality += centrality(dest, attr_mat, K)
        X1.append(attr_mat.loc[dest]['total_attr_log']/np.mean(attr_mat['total_attr_log']))
        X2.append(dist_matrix.loc[origin][dest]/ np.mean(dist_matrix.loc[origin]))
        X3.append(centrality(dest, attr_mat, K)/(total_centrality/len(dests)))
    var_table = pd.DataFrame()
    X1 = [x + 1 for x in X1]
    X3 = [x + 1 for x in X3]
    var_table['x1'] = np.nan_to_num(np.log(X1))
    var_table['x2'] = np.nan_to_num(np.log(X2))
    var_table['x3'] = np.nan_to_num(np.log(X3))
    return var_table

#### Reproduce Table 2 in the paper

In [47]:
# Y value used for both table 2 and 3
Y_res = []
for place in places:
    for file in pmatrix_all:
        Y = read_actual(file, place)
        log_Y = np.nan_to_num(np.log(Y))
        Y_res = np.append(Y_res, np.round(log_Y,10))

In [48]:
def var_tbl(val):
    var_table = []
    for place in places:
        for i in range(1,13):
            tbl = log_transform_x(place,2,i,val)
            var_table.append(tbl)
    df_var_tbl = pd.concat(var_table)
    return df_var_tbl

def clear_var_tbl(df):
    df['Y'] = Y_res
    df = df[df.Y > 0]
    df = df[df.x1 != 0]
    return df

def return_tbl2_results(df):
    df = clear_var_tbl(df)
    X = df[['x1', 'x2','x3']]
    Y = df['Y']
    model = sm.OLS(Y,X).fit()
    r2 = round(model.rsquared,3)
    aic = round(model.aic,1)
    return [r2,aic]

In [49]:
tbl2_results=[]
for val in range(1,4):
    df_tbl2 = var_tbl(val)
    tbl2_results.append(return_tbl2_results(df_tbl2))
    
def generate_tbl2(tbl2_results):
    df_tbl2 = pd.DataFrame(columns=['Aj1','Aj2','Aj3'])
    df_tbl2['Aj1'] = tbl2_results[0]
    df_tbl2['Aj2'] = tbl2_results[1]
    df_tbl2['Aj3'] = tbl2_results[2]
    df_tbl2 = df_tbl2.T
    df_tbl2.columns =['R2', 'AIC']
    df_tbl2['delta_AIC'] = df_tbl2['AIC'] - min(df_tbl2['AIC'])
    df_tbl2['w_i'] = np.exp(-0.5*df_tbl2['delta_AIC'])/sum(np.exp(-0.5*df_tbl2['delta_AIC']))
    return df_tbl2

park_tbl2 = generate_tbl2(tbl2_results)
park_tbl2

,R2,AIC,delta_AIC,w_i
Aj1,0.743,724.6,14.9,0.000581
Aj2,0.741,728.1,18.4,0.000101
Aj3,0.753,709.7,0.0,0.999318


#### Reproduce Table 3 in the paper

In [13]:
def var_tbl3(val, time):
    var_table = []
    for place in places:
        for i in range(1,13):
            if time == True:
                tbl = log_transform_x(place,2,i,val)
            else:
                tbl = log_transform_x_NT(place,2)
            var_table.append(tbl)
    df_var_tbl = pd.concat(var_table)
    return df_var_tbl

def return_tbl3_results(df, neighbor):
    df = clear_var_tbl(df)
    if neighbor == True:
        X = df[['x1', 'x2','x3']]
    else:
        X = df[['x1', 'x2']]
    Y = df['Y']
    model = sm.OLS(Y,X).fit()
    r2 = round(model.rsquared,3)
    aic = round(model.aic,1)
    return [r2,aic]

In [14]:
def model_selection(neighbor, time):
    df_tbl3 = var_tbl3(3,time) #Aj3 is used here
    return return_tbl3_results(df_tbl3, neighbor)
    
def generate_tbl3():
    df_tbl3 = pd.DataFrame(columns=['SA_model','SA_model_no_N','SA_model_no_T','Huff_model'])
    df_tbl3['SA_model'] = model_selection(neighbor=True, time=True)
    df_tbl3['SA_model_no_N'] = model_selection(neighbor=False, time=True)
    df_tbl3['SA_model_no_T'] = model_selection(neighbor=True, time=False)
    df_tbl3['Huff_model'] = model_selection(neighbor=False, time=False)
    df_tbl3 = df_tbl3.T
    df_tbl3.columns =['R2', 'AIC']
    df_tbl3['delta_AIC'] = df_tbl3['AIC'] - min(df_tbl3['AIC'])
    df_tbl3['w_i'] = np.exp(-0.5*df_tbl3['delta_AIC'])/sum(np.exp(-0.5*df_tbl3['delta_AIC']))
    return df_tbl3

park_tbl3 = generate_tbl3()
park_tbl3

,R2,AIC,delta_AIC,w_i
SA_model,0.753,709.7,0.0,9.859364e-01
SA_model_no_N,0.746,718.2,8.5,1.406363e-02
SA_model_no_T,0.744,748.5,38.8,3.702848e-09
Huff_model,0.738,755.8,46.1,9.624121e-11


#### Reproduce Table 4 in the paper

In [15]:
def Y_res_place(place):
    Y_res = []
    for file in pmatrix_all:
        Y = read_actual(file, place)
        log_Y = np.nan_to_num(np.log(Y))
        Y_res = np.append(Y_res, np.round(log_Y,10))
    return Y_res

def var_table_place(place):
    var_table = []
    for i in range(1,13):
        tbl = log_transform_x(place,2,i,3) #Aj3
        var_table.append(tbl)
    return pd.concat(var_table)

def clear_var_tbl_place(df, place):
    df['Y'] = Y_res_place(place)
    df = df[df.Y > 0]
    df = df[df.x1 != 0]
    return df

def return_tbl_4_5_results(df,place):
    df = clear_var_tbl_place(df,place)
    num_obs = len(df)
    X = df[['x1', 'x2','x3']]
    Y = df['Y']    
    model = sm.OLS(Y,X).fit()    
    alpha = round(model.params[0],4)
    alpha_p = round(model.pvalues[0],3)
    beta = round(model.params[1],4)
    beta_p = round(model.pvalues[1],3)
    theta = round(model.params[2],4)
    theta_p = round(model.pvalues[2],3)
    mse = round(model.mse_resid,3)
    r2 = round(model.rsquared,3)    
    return [place, num_obs, alpha, alpha_p, beta, beta_p, theta, theta_p, mse, r2]

def mark_significance(lst):
    p_val = []
    for item in lst:
        if item <= 0.001:
            p_val.append('***')
        elif item > 0.001 and item <= 0.01:
            p_val.append('**')
        elif item > 0.01 and item <= 0.05:
            p_val.append('*')
        else:
            p_val.append(' ')
    return p_val

In [16]:
tbl_4_5_results = []
for place in places:
    var_table = var_table_place(place)
    tbl_4_5_results.append(return_tbl_4_5_results(var_table,place))

def generate_tbl_4_5(tbl_results):
    df_tbl_4_5 = pd.DataFrame(data=tbl_results,columns=['Place','num_obs','alpha','alpha_p','beta','beta_p','theta','theta_p','MSE','R2'])
    df_tbl_4_5 = df_tbl_4_5[df_tbl_4_5['num_obs'] >= 30]
    df_tbl_4_5['alpha_sig'] = mark_significance(df_tbl_4_5['alpha_p'])
    df_tbl_4_5['beta_sig'] = mark_significance(df_tbl_4_5['beta_p'])
    df_tbl_4_5['theta_sig'] = mark_significance(df_tbl_4_5['theta_p'])
    df_tbl_4_5 = df_tbl_4_5[['Place','alpha','alpha_sig','beta','beta_sig','theta','theta_sig','MSE','R2']]
    return df_tbl_4_5

generate_tbl_4_5(tbl_4_5_results)

,Place,alpha,alpha_sig,beta,beta_sig,theta,theta_sig,MSE,R2
1,Bass Harbor,0.8863,*,-0.8608,*,0.1155,,0.273,0.731
3,Northeast Harbor,0.1684,,-0.1137,,0.4079,*,0.208,0.838
4,Bar Harbor,1.3226,***,0.2359,,-0.0224,,0.322,0.739
6,Cadillac Mountain,0.6601,,-0.0218,,0.1907,,0.360,0.707
8,Bubble Rock,0.1722,,-0.4898,*,0.3994,*,0.322,0.791
9,Jordan Pond,1.5456,***,-0.0670,,-0.0133,,0.203,0.886
10,Boulder Beach,2.0496,***,0.3590,*,-0.3446,,0.334,0.751
11,Thunder Hole,1.2109,**,-0.1355,,0.0232,,0.301,0.782
12,Sand Beach,2.1061,***,0.0374,,-0.3318,,0.397,0.742


#### Reproduce Table 7 in the paper

In [18]:
def var_tbl_7(K, time):
    var_table = []
    if time == 0: #all-time
        month_range = range(1,13)
    if time == 1: # summer
        month_range = range(5,10)
    if time == 2: #non-summer
        month_range = [1,2,3,4,10,11,12]
    for place in places:
        for i in month_range:
            tbl = log_transform_x(place,K,i,3) #Aj3
            var_table.append(tbl)
    df_var_tbl = pd.concat(var_table)
    return df_var_tbl

def clear_var_tbl(df, Y_val):
    df['Y'] = Y_val
    df = df[df.Y > 0]
    df = df[df.x1 != 0]
    return df

def return_tbl7_results(df,Y_val):
    df = clear_var_tbl(df,Y_val)
    X = df[['x1', 'x2','x3']]
    Y = df['Y']
    model = sm.OLS(Y,X).fit()
    mse = round(model.mse_resid,6)
    r2 = round(model.rsquared,3)
    return [mse,r2]

In [19]:
Y_res_all,Y_res_summer,Y_res_non_summer = [],[],[]
for place in places:
    for file in pmatrix_all:
        Y = read_actual(file, place)
        log_Y = np.nan_to_num(np.log(Y))
        Y_res_all = np.append(Y_res_all, np.round(log_Y,10))
    for file in pmatrix_summer:
        Y = read_actual(file, place)
        log_Y = np.nan_to_num(np.log(Y))
        Y_res_summer = np.append(Y_res_summer, np.round(log_Y,10))
    for file in pmatrix_non_summer:
        Y = read_actual(file, place)
        log_Y = np.nan_to_num(np.log(Y))
        Y_res_non_summer = np.append(Y_res_non_summer, np.round(log_Y,10))

In [20]:
tbl7_all,tbl7_summer,tbl7_non_summer,tbl7_results=[],[],[],[]
for K in [2,3,5]:
    df_all = var_tbl_7(K,0)
    df_summer = var_tbl_7(K,1)
    df_non_summer = var_tbl_7(K,2)   
    tbl7_all.append(return_tbl7_results(df_all,Y_res_all))
    tbl7_summer.append(return_tbl7_results(df_summer,Y_res_summer))
    tbl7_non_summer.append(return_tbl7_results(df_non_summer,Y_res_non_summer))    
tbl7_results = tbl7_all+tbl7_summer+tbl7_non_summer

def generate_tbl7(results):
    df_tbl7 = pd.DataFrame(data=results)
    df_tbl7.columns =['MSE', 'R2']
    df_tbl7['Time'] = ["All_year"]*3+['Summer']*3+['Non_summer']*3
    df_tbl7['K'] = [2,3,5]*3
    df_tbl7 = df_tbl7.set_index(['Time', 'K'])    
    return df_tbl7

park_tbl7 = generate_tbl7(tbl7_results)
park_tbl7

MSE     R2
Time       K                 
All_year   2  0.351958  0.753
           3  0.355672  0.750
           5  0.360020  0.747
Summer     2  0.239118  0.780
           3  0.241003  0.778
           5  0.241888  0.777
Non_summer 2  0.470965  0.766
           3  0.479230  0.762
           5  0.490113  0.757